# Introducing fastpages
> An easy to use blogging platform with extra features for <a href="https://jupyter.org/">Jupyter Notebooks</a>.

- toc: true 
- badges: true
- comments: true
- sticky_rank: 1
- author: Jeremy Howard & Hamel Husain
- image: images/fastpages.png
- categories: [fastpages, jupyter]

![](https://github.com/fastai/fastpages/raw/master/images/diagram.png "https://github.com/fastai/fastpages")

We are very pleased to announce the immediate availability of [fastpages](https://github.com/fastai/fastpages). `fastpages` is a platform which allows you to create and host a blog for free, with no ads and many useful features, such as:
